In [ ]:
import pygame
import cv2
import numpy as np
import pygame
from pygame import mixer
from playsound import playsound
import os
from twilio.rest import Client 
pygame.mixer.init()

# Define your Twilio credentials
account_sid = '@Twilio account_sid'
auth_token = '@Twilio auth_token'
twilio_phone_number = '@twilio_phone_number'
user_phone_number = '@user_phone_number'

# Define the file paths for the sound files
crackers_sound_file = "C:/Users/91630/OneDrive/Desktop/new notebook/scarecrow/Scarecrow/sounds/CrackersSound.mp3"
lion_roar_sound_file = "C:/Users/91630/OneDrive/Desktop/new notebook/scarecrow/Scarecrow/sounds/LionRoarSound.mp3"

# Initialize the Twilio client
client = Client(account_sid, auth_token)

cap = cv2.VideoCapture(0)
wht = 320
confThreshold = 0.5
nmsThreshold = 0.3
classesFile = 'coco.names'
classNames = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
modelConfi = 'yolov3.cfg'
modelwei = 'yolov3.weights'
net = cv2.dnn.readNetFromDarknet(modelConfi, modelwei)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

def send_notification(animal):
    message = client.messages.create(
        body=f"An {animal} has entered the field!",
        from_=twilio_phone_number,
        to=user_phone_number
    )
    print(f"Notification sent: {message.sid}")

def play_sound(sound_file):
    try:
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play()
    except pygame.error as e:
        print(f"Error playing sound: {e}")

def findobj(outputs, img):
    ht, wt, ct = img.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w, h = int(det[2] * wt), int(det[3] * ht)
                x, y = int((det[0] * wt) - w / 2), int((det[1] * ht) - h / 2)
                bbox.append([x, y, w, h])
                classIds.append(classId)
                confs.append(float(confidence))
    indices = cv2.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)
    for i in indices:
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 2)
        cv2.putText(img, f'{classNames[classIds[i]].upper()} {int(confs[i] * 100)}%', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 2))
        x = classNames[classIds[i]]
        if x in ["bird", "horse", "truck", "elephant", "bear", "zebra", "giraffe"]:
            send_notification(x)
            play_sound(crackers_sound_file)
        elif x in ["cat", "dog", "sheep"]:
            send_notification(x)
            play_sound(lion_roar_sound_file)

while True:
    outputnames = []
    success, img = cap.read()
    blob = cv2.dnn.blobFromImage(img, 1/255, (wht, wht), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    outputnames = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputnames)
    findobj(outputs, img)
    cv2.imshow('image', img)
    k = cv2.waitKey(50)
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()


pygame 2.5.0 (SDL 2.28.0, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Notification sent: SMab6bfc8c56d3bf91b37e0f648d0ec4cc
Error playing sound: No such file or directory: 'C:/Users/91630/OneDrive/Desktop/new notebook/scarecrow/Scarecrow/sounds/LionRoarSound.mp3'.
Notification sent: SMef0ee97d762c084c5f8c711977585b59
Error playing sound: No such file or directory: 'C:/Users/91630/OneDrive/Desktop/new notebook/scarecrow/Scarecrow/sounds/LionRoarSound.mp3'.
Notification sent: SM883c746cfb14a0735ccb5f4c46d06a8d
Error playing sound: No such file or directory: 'C:/Users/91630/OneDrive/Desktop/new notebook/scarecrow/Scarecrow/sounds/LionRoarSound.mp3'.
Notification sent: SM2ec2018580268bb69185af437d27d99a
Error playing sound: No such file or directory: 'C:/Users/91630/OneDrive/Desktop/new notebook/scarecrow/Scarecrow/sounds/LionRoarSound.mp3'.
Notification sent: SM78f40f148c931c7fdd1d212fa8b243d0
Error playing sound: No such file or directory: 'C:/U